In [19]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from enum import Enum

class ProposalType(Enum):
    GLOBAL = 'global'
    LOCAL = 'local'
    FREEFORM = 'freeform'
    ALGORITHM = 'algorithm'
    
class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'
    
%matplotlib inline

In [20]:
dms = dict([(sc, DataManager(stack='MD594', section=sc, segm_params_id='tSLIC200', load_mask=False)) 
			for sc in range(47, 185)])

In [21]:
for dm in dms.itervalues():
    dm.reload_labelings()

In [24]:
from matplotlib.path import Path

In [22]:
from shapely.geometry import Polygon, Point

In [23]:
label_examples = defaultdict(list) # cluster, dedges, sig, stack, sec, proposal_type, labeling username, timestamp

for sec, dm in dms.iteritems():
    
    res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    
    if res is None:
        continue
    
    dm.load_multiple_results(['texHist', 'spCentroids', 'edgeMidpoints', 'edgeEndpoints'])
        
    usr, ts, suffix, result = res
    
    for props in result:
        if props['type'] == ProposalType.FREEFORM:
#             pg = Polygon(props['vertices'])
#             cl = np.where([pg.contains(Point(x,y)) for y,x in dm.sp_centroids])[0]
            pp = Path(props['vertices'])
            cl = np.where([pp.contains_point((x,y)) for y,x in dm.sp_centroids])[0]
        
            de = dm.find_boundary_dedges_ordered(cl)
            sig = dm.compute_cluster_score(cl, method='rc-mean')[0]
            label_examples[props['label']].append((cl, de, sig, dm.stack, dm.slice_ind, 
                                                   ProposalType.FREEFORM, usr, ts))            
        else:
            label_examples[props['label']].append((props['sps'], props['dedges'], props['sig'], 
                                      dm.stack, dm.slice_ind, props['type'], usr, ts))

label_examples.default_factory = None

dedges missing 22 61%


In [25]:
label_texture = {}

for label, proposals in label_examples.iteritems():
    w = []
    for prop in proposals:
        cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
        w.append(dms[sec].texton_hists[cluster].mean(axis=0))
        
    label_texture[label] = np.mean(w, axis=0)

In [26]:
def fit_ellipse_to_points(pts):

    pts = np.array(list(pts) if isinstance(pts, set) else pts)

    c0 = pts.mean(axis=0)

    coords0 = pts - c0

    U,S,V = np.linalg.svd(np.dot(coords0.T, coords0)/coords0.shape[0])
    v1 = U[:,0]
    v2 = U[:,1]
    s1 = np.sqrt(S[0])
    s2 = np.sqrt(S[1])

    return v1, v2, s1, s2, c0

In [ ]:
# label_position = {}

# for label, proposals in label_examples.iteritems():
#     all_pts = []
#     for prop in proposals:
#         cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
#         pts = np.vstack([ dms[sec].sp_coords[sp][:, ::-1] for sp in cluster])
#         all_pts.append(pts)
        
#     all_pts = np.vstack(all_pts)
    
#     ell = fit_ellipse_to_points(all_pts)
    
#     label_position[label] = ell

In [27]:
label_position = defaultdict(lambda: {})

for label, proposals in label_examples.iteritems():

    for prop in proposals:
        cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
        
#         dms[sec].load_multiple_results(['spCoords'])
#         pts = np.vstack([ dms[sec].sp_coords[sp][:, ::-1] for sp in cluster])
        
        ell = fit_ellipse_to_points(dms[sec].sp_centroids[cluster][:,::-1])
        label_position[label][sec] = ell
        
label_position.default_factory=None

In [28]:
label_position.keys()

['Tz', 'VLL', '7n', '5N', 'RtTg', '7N', 'LSO', 'Pn', 'LC']

In [29]:
label_polygon = defaultdict(lambda: {})

for label, proposals in label_examples.iteritems():

    for prop in proposals:
        cluster, dedges, sig, stack, sec, proposal_type, username, timestamp = prop
        vs = dms[sec].vertices_from_dedges(dedges)
        polygon = Polygon(vs)
        label_polygon[label][sec] = polygon
        
#         pts = np.vstack([ dms[sec].sp_coords[sp][:, ::-1] for sp in cluster])
#         label_coords[label][sec] = pts

label_polygon.default_factory=None

In [30]:
pickle.dump(label_examples, open(os.environ['GORDON_RESULT_DIR']+'/database/label_examples.pkl', 'w'))
pickle.dump(label_position, open(os.environ['GORDON_RESULT_DIR']+'/database/label_position.pkl', 'w'))
pickle.dump(label_polygon, open(os.environ['GORDON_RESULT_DIR']+'/database/label_polygon.pkl', 'w'))
pickle.dump(label_texture, open(os.environ['GORDON_RESULT_DIR']+'/database/label_texture.pkl', 'w'))